In [2]:
!pip install medspacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.4/111.4 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.4/67.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.5/575.5 kB 31.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 22.7 MB/s eta 0:00:00
  Created wheel for medspacy: filename=medspacy-1.1.2-py3-none-any.whl size=147920 sha25

In [3]:
import numpy as np 
import pandas as pd
import os

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 10})

In [4]:
df = pd.read_csv('/kaggle/input/morticd10/codes.csv')

#structuring
df.columns = ['prefix_codes', 'decimal', 'codes', 'description', 'additional_des', 'label']

In [5]:
category_distribution = df['prefix_codes'].value_counts()
lower_threshold = 5
upper_threshold = 10
filtered_categories = category_distribution[(category_distribution > lower_threshold) & (category_distribution < upper_threshold)]

print(filtered_categories)

K43      9
R8761    9
K28      9
K27      9
F317     9
        ..
D55      6
D48      6
B36      6
Y9241    6
C7B0     6
Name: prefix_codes, Length: 2243, dtype: int64


In [6]:
new_df = df[df['prefix_codes'].isin(filtered_categories.index)]

In [7]:
new_df = new_df.sample(frac=1, random_state=23)
print(f"Size: {len(new_df)}")
new_df.head()

Size: 15036


,prefix_codes,decimal,codes,description,additional_des,label
28706,S32463,D,S32463D,"Displ assoc transv/post fx unsp acetab, 7thD",Displaced associated transverse-posterior frac...,Displaced associated transverse-posterior frac...
18763,N88,0,N880,Leukoplakia of cervix uteri,Leukoplakia of cervix uteri,Other noninflammatory disorders of cervix uteri
4627,G11,8,G118,Other hereditary ataxias,Other hereditary ataxias,Hereditary ataxia
3707,E7524,1,E75241,Niemann-Pick disease type B,Niemann-Pick disease type B,Niemann-Pick disease
31511,S42456,A,S42456A,"Nondisp fx of lateral condyle of unsp humerus,...",Nondisplaced fracture of lateral condyle of un...,Nondisplaced fracture of lateral condyle of un...


In [8]:
new_df.iloc[0].additional_des, new_df.iloc[0].description

('Displaced associated transverse-posterior fracture of unspecified acetabulum, subsequent encounter for fracture with routine healing',
 'Displ assoc transv/post fx unsp acetab, 7thD')

In [9]:
from sklearn.model_selection import train_test_split

X = df[['additional_des']]
y = df['prefix_codes']

# Split the dataset into a train and temporary set
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the temporary set into validation and train set
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

In [10]:
import medspacy
from medspacy.ner import TargetRule
from medspacy.visualization import visualize_ent

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso